In [1]:
from datasets import load_dataset

In [4]:
eli5 = load_dataset("eli5", split="train_asks[:1000]")

Downloading:   0%|          | 0.00/3.50k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/576M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/21.1M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/53.0M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/286M [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/17.7M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/330M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/18.7M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/36.2M [00:00<?, ?B/s]

Dataset eli5 downloaded and prepared to /home/edan/.cache/huggingface/datasets/eli5/LFQA_reddit/1.0.0/17574e5502a10f41bbd17beba83e22475b499fa62caa1384a3d093fc856fe6fa. Subsequent calls will reuse this data.


In [5]:
eli5 = eli5.train_test_split(test_size=0.2)

In [6]:
eli5["train"][0]

{'q_id': '10fpo5',
 'title': 'Evolution: Good learning resources? Combatting rampant misunderstandings of it.',
 'selftext': '',
 'document': '',
 'subreddit': 'askscience',
 'answers': {'a_id': ['c6d2jii'],
  'text': ["Here's a good place for you to start  \n_URL_0_"],
  'score': [2]},
 'title_urls': {'url': []},
 'selftext_urls': {'url': []},
 'answers_urls': {'url': ['http://wiki.ironchariots.org/index.php?title=Evolution']}}

In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilgpt2")

In [8]:
eli5 = eli5.flatten()
eli5["train"][0]

{'q_id': '10fpo5',
 'title': 'Evolution: Good learning resources? Combatting rampant misunderstandings of it.',
 'selftext': '',
 'document': '',
 'subreddit': 'askscience',
 'answers.a_id': ['c6d2jii'],
 'answers.text': ["Here's a good place for you to start  \n_URL_0_"],
 'answers.score': [2],
 'title_urls.url': [],
 'selftext_urls.url': [],
 'answers_urls.url': ['http://wiki.ironchariots.org/index.php?title=Evolution']}

In [9]:
def preprocess_function(examples):
    return tokenizer([" ".join(x) for x in examples["answers.text"]])

In [10]:
tokenized_eli5 = eli5.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns=eli5["train"].column_names,
)

Map (num_proc=4):   0%|          | 0/800 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (2193 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1173 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1139 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (2068 > 1024). Running this sequence through the model will result in indexing errors


Map (num_proc=4):   0%|          | 0/200 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1076 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1103 > 1024). Running this sequence through the model will result in indexing errors


In [12]:
tokenized_eli5['train'][0]

{'input_ids': [4342,
  338,
  257,
  922,
  1295,
  329,
  345,
  284,
  923,
  220,
  220,
  198,
  62,
  21886,
  62,
  15,
  62],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [18]:
tokenized_eli5['train'].keys()

AttributeError: 'Dataset' object has no attribute 'keys'

In [29]:
block_size = 128

def group_texts(examples):
    # Concatenate all texts.
    print(list(examples.keys()))
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # Split by chunks of block_size.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [31]:
lm_dataset = tokenized_eli5.map(group_texts, batched=True, num_proc=4)

Map (num_proc=4):   0%|          | 0/800 [00:00<?, ? examples/s]

['input_ids', 'attention_mask']['input_ids', 'attention_mask']

['input_ids', 'attention_mask']
['input_ids', 'attention_mask']


TypeError: can only concatenate list (not "int") to list

In [16]:
lm_dataset['train'][0]

{'input_ids': [4342,
  338,
  257,
  922,
  1295,
  329,
  345,
  284,
  923,
  220,
  220,
  198,
  62,
  21886,
  62,
  15,
  62,
  5297,
  13,
  632,
  338,
  2192,
  625,
  47356,
  1335,
  2233,
  284,
  262,
  8722,
  286,
  1180,
  26336,
  1200,
  17211,
  2761,
  357,
  28950,
  3519,
  284,
  1641,
  2761,
  828,
  9963,
  21228,
  31170,
  8967,
  11,
  290,
  22822,
  11,
  355,
  880,
  355,
  257,
  2187,
  26891,
  286,
  584,
  4673,
  11916,
  588,
  38949,
  7587,
  2761,
  13,
  1475,
  22428,
  515,
  416,
  262,
  1109,
  326,
  340,
  318,
  1690,
  7317,
  14641,
  290,
  5716,
  416,
  4165,
  1337,
  17206,
  2427,
  286,
  22447,
  11,
  475,
  340,
  318,
  257,
  1103,
  1517,
  13,
  1875,
  1148,
  27841,
  257,
  1103,
  1517,
  30,
  198,
  198,
  5297,
  13,
  628,
  1875,
  1148,
  326,
  8967,
  18269,
  30,
  198,
  198,
  1026,
  338,
  517,
  3716,
  621,
  661,
  6939,
  618,
  340,
  717,
  2627,
  257,
  8661,
  3721],
 'attention_mask': [1,
  1